In [ ]:
import os, sys

sys.path.append(os.path.abspath('..'))

import json
from PyQt5.QtWidgets import QApplication
from src.gui.chunk_manager import ChunkManager

from src.utils import  chunkTextForTTS, create_audio, create_project_folder
from src.enums import ElevenLabsTTSModel

# if not specified, will use default model from config/config.yaml
tts_model = ElevenLabsTTSModel.Turbo_v25.value

script_text = """"""
script_location = "input-test.txt"

# maybe make a copy of this notebook for F5 TTS, then combine the 2 later

def save_chunks(chunks, project_folder):
	chunks_name = os.path.join(project_folder, 'tts-chunks.json')
	with open(chunks_name, 'w') as f:
		f.write(json.dumps(chunks))

input_exists = os.path.exists(script_location)
if not script_text and not input_exists:
	raise SystemExit("Please provide a script or a script location")

if not script_text and script_location:
	with open(script_location, 'r') as f:
		script_text = f.read()

# make the text chunks
chunks = chunkTextForTTS(script_text)
project_folder = create_project_folder(reuse=True)

# save the chunks
chunks_name = os.path.join(project_folder, 'tts-chunks.json')
if not os.path.exists(chunks_name):
	print(f"Saving text chunks to {chunks_name}")
	save_chunks(chunks, project_folder)
else:
	# compare with existing chunks
	with open(chunks_name, 'r') as f:
		existing_chunks = eval(f.read())
		print('Comparing with existing text chunks')
		if existing_chunks != chunks:
			print('Chunks have changed, updating')
			save_chunks(chunks, project_folder)

# generate the tts audio files
skipped = 0
for i, chunk in enumerate(chunks):
	chunk_type = chunk['type']
	audio_name = f'{i}_{chunk_type}.mp3'
	chunk['audio'] = audio_name
	audio_exists = os.path.exists(os.path.join(project_folder, audio_name))

	if audio_exists:
		skipped += 1
		continue

	content = chunk['content']
	create_audio(content, audio_name, project_folder, tts_model)

save_chunks(chunks, project_folder)

if skipped > 0:
	print(f"Skipped generating {skipped} audio files")

# open ui to review chunks
app = QApplication(sys.argv)
ex = ChunkManager(chunks, project_folder)
ex.show()
app.exec_()
%gui qt

In [2]:
# elevenlabs.getModels()

In [ ]:
# Standalone GUI:
# use this cell if chunks are already saved and generated

import os
import sys
from PyQt5.QtWidgets import QApplication
from src.gui.chunk_manager import ChunkManager
from src.utils import create_project_folder

project_folder = create_project_folder(reuse=True)
chunks_name = os.path.join(project_folder, 'tts-chunks.json')
with open(chunks_name, 'r') as f:
	chunks = eval(f.read())

app = QApplication(sys.argv)
ex = ChunkManager(chunks, project_folder)
ex.show()
app.exec_()
%gui qt

List of things to do:

- Need buttons in the window, to add a chunk above or below the selected chunk
- Add a "Chunks" title and remove "Chunk"s from the list
- Idea for having options show when selecting text:
  - I'm sure we can get an event on text select (we can get the selected text right?)
  - When there's text selected, show an area maybe above the text box with buttons to do things like to split the text into a new chunk, or to merge the selected chunk with the one above or below it

It also feels like I might as well convert this whole this into a script.
